In [39]:
from pathlib import Path

import pandas as pd

from source.utils import POST_PROC_DIR, corners, timestamp_today, confirm_dir, print_md_table, print_iter
from source.utils.associate import AM_DF_DIR, TOP_AM_DIR, adjust_assoc_columns
from source.utils.sample import sample_pickle as sp

K=9
BK = max(K+2, 10)
DATE=timestamp_today()
FOCUS_ORIG = ['f', 'E11', 'unexpected_f',
              'am_p1_given2', 'conservative_log_ratio',
              'am_log_likelihood', 
            #   't_score',
            #   'mutual_information', 'am_odds_ratio_disc',
              'N', 'f1', 'f2', 'l1', 'l2']
FOCUS = adjust_assoc_columns(FOCUS_ORIG)
pd.set_option("display.float_format", '{:,.2f}'.format)
pd.set_option("display.max_colwidth", 80)
try:
    adv_am = pd.read_csv(
    TOP_AM_DIR / f'Top{K}_NEG-ADV_combined.35f-7c_{DATE}.csv').set_index('adv')
except FileNotFoundError: 
    DATE=DATE[:-1]+str(int(DATE[-1])-1)
    adv_am = pd.read_csv(
    TOP_AM_DIR / f'Top{K}_NEG-ADV_combined.35f-7c_{DATE}.csv').set_index('adv')
# loaded_cols = adv_am.columns.to_list()
HITS_SAMPLE_PATH = TOP_AM_DIR.joinpath(f'top{K}adv_sample-{K}-hit-tables_{DATE}.pkl.gz')
if not HITS_SAMPLE_PATH.is_file(): 
    yesterday=DATE[:-1]+str(int(DATE[-1])-1)
    HITS_SAMPLE_PATH = TOP_AM_DIR.joinpath(f'top{K}adv_sample-{K}-hit-tables_{yesterday}.pkl.gz')
    if not HITS_SAMPLE_PATH.is_file(): 
        HITS_SAMPLE_PATH = list(TOP_AM_DIR.glob(f'top{K}adv_sample-{K}-hit-tables_*.pkl.gz'))[0]


In [40]:
def nb_print_table(df, n_dec:int=2, 
                   adjust_columns:bool=True, 
                   outpath:Path=None) -> None: 
    _df = df.copy()
    if adjust_columns: 
        _df = adjust_assoc_columns(_df)
    _df.columns = [f'`{c}`' for c in _df.columns]
    _df.index = [f'**{r}**' for r in _df.index ]
    table = _df.to_markdown(floatfmt=f',.{n_dec}f', intfmt=',')
    if outpath:
        outpath.write_text(table)

    print(f'\n{table}\n')

In [41]:
adv_am

,key_SET,f_SET,dP1_SET,LRC_SET,G2_SET,N_SET,f1_SET,f2_SET,exp_f_SET,unexp_f_SET,...,mean_G2,mean_N,mean_f1,mean_f2,mean_expF,mean_unexpF,ratio_f_MIR,ratio_N_MIR,ratio_f1_MIR,ratio_f2_MIR
adv,,,,,,,,,,,,,,,,,,,,,
exactly,NEGany~exactly,43635,0.67,5.90,"214,404.20",86330752,3226213,61599,"2,301.98","41,333.02",...,"108,171.83","44,181,417.00","1,760,088.00","31,356.50","1,231.57","20,992.43",0.02,0.02,0.09,0.02
necessarily,NEGany~necessarily,42708,0.72,6.23,"219,003.46",86330752,3226213,56694,"2,118.68","40,589.32",...,"110,346.18","44,181,417.00","1,760,088.00","29,187.50","1,180.93","20,658.57",0.02,0.02,0.09,0.03
before,NEG~before,311,0.38,3.65,"1,062.13",86330752,3226213,748,27.95,283.05,...,"1,071.32","44,181,417.00","1,760,088.00",521.00,35.24,265.26,0.93,0.02,0.09,0.39
that,NEGany~that,165411,0.63,5.62,"781,016.11",86330752,3226213,250392,"9,357.24","156,053.76",...,"394,324.16","44,181,417.00","1,760,088.00","128,932.00","5,219.08","79,655.42",0.03,0.02,0.09,0.03
remotely,NEGany~remotely,5679,0.22,3.03,"13,354.33",86330752,3226213,22194,829.40,"4,849.60",...,"8,682.08","44,181,417.00","1,760,088.00","12,455.50",611.22,"3,151.28",0.33,0.02,0.09,0.12
yet,NEGany~yet,52546,0.48,4.74,"209,055.78",86330752,3226213,101707,"3,800.83","48,745.17",...,"104,649.01","44,181,417.00","1,760,088.00","51,308.00","1,966.16","24,466.84",0.01,0.02,0.09,0.01
ever,NEGany~ever,5967,0.01,0.28,353.58,86330752,3226213,124592,"4,656.05","1,310.95",...,"7,846.96","44,181,417.00","1,760,088.00","64,885.50","2,702.62","2,639.88",0.79,0.02,0.09,0.04
immediately,NEGany~immediately,57319,0.52,4.96,"239,462.58",86330752,3226213,103177,"3,855.76","53,463.24",...,"119,821.89","44,181,417.00","1,760,088.00","52,309.50","2,032.18","26,830.82",0.01,0.02,0.09,0.01
any,NEGany~any,15492,0.13,2.28,"23,683.00",86330752,3226213,94152,"3,518.50","11,973.50",...,"13,097.13","44,181,417.00","1,760,088.00","47,833.00","1,868.76","6,418.24",0.07,0.02,0.09,0.02


In [42]:
adv_list = adv_am.index.to_list()
print_iter(adv_list, header=f'Top {K} Most Negative Adverbs',bullet = '1.')


Top 9 Most Negative Adverbs
1. exactly
1. necessarily
1. before
1. that
1. remotely
1. yet
1. ever
1. immediately
1. any
1. particularly
1. terribly
1. inherently
1. altogether
1. only


## Load AM table for `adv~adj` comparison (bigram composition)

In [43]:
adv_adj_am = adjust_assoc_columns(
    pd.read_pickle(AM_DF_DIR.joinpath('adv_adj/RBXadj/extra/AdvAdj_frq-thrMIN-7.35f_min200x_extra.pkl.gz'))
                                  ).filter(items=FOCUS).sort_values('LRC', ascending = False)
adv_adj_am = adv_adj_am.loc[adv_adj_am.l1.isin(adv_list), :]
nb_print_table(adv_adj_am.head(20))
adv_adj_am


|                             |   `f` |   `exp_f` |   `unexp_f` |   `dP1` |   `LRC` |      `G2` |        `N` |    `f1` |   `f2` | `l1`        | `l2`         |
|:----------------------------|------:|----------:|------------:|--------:|--------:|----------:|-----------:|--------:|-------:|:------------|:-------------|
| **yet~unborn**              |   463 |      0.75 |      462.25 |    0.73 |   10.47 |  5,505.48 | 86,330,753 | 101,707 |    635 | yet         | unborn       |
| **only~half-joking**        |   209 |      1.17 |      207.83 |    0.96 |    9.48 |  2,116.01 | 86,330,753 | 464,174 |    217 | only        | half-joking  |
| **inherently~governmental** |   295 |      0.60 |      294.40 |    0.32 |    8.96 |  3,178.72 | 86,330,753 |  55,088 |    933 | inherently  | governmental |
| **immediately~accretive**   |   243 |      0.68 |      242.32 |    0.43 |    8.65 |  2,499.70 | 86,330,753 | 103,177 |    565 | immediately | accretive    |
| **exactly~alike**           | 3,040 |      

,f,exp_f,unexp_f,dP1,LRC,G2,N,f1,f2,l1,l2
key,,,,,,,,,,,
yet~unborn,463,0.75,462.25,0.73,10.47,"5,505.48",86330753,101707,635,yet,unborn
only~half-joking,209,1.17,207.83,0.96,9.48,"2,116.01",86330753,464174,217,only,half-joking
inherently~governmental,295,0.60,294.40,0.32,8.96,"3,178.72",86330753,55088,933,inherently,governmental
immediately~accretive,243,0.68,242.32,0.43,8.65,"2,499.70",86330753,103177,565,immediately,accretive
exactly~alike,3040,9.46,"3,030.54",0.23,8.55,"29,939.32",86330753,61599,13261,exactly,alike
...,...,...,...,...,...,...,...,...,...,...,...
only~high,327,"3,177.51","-2,850.51",-0.00,-2.86,"-4,245.38",86330753,464174,590978,only,high
only~sure,339,"4,543.20","-4,204.20",-0.01,-3.34,"-6,708.28",86330753,464174,844981,only,sure
ever~many,206,"3,193.77","-2,987.77",-0.00,-3.42,"-4,923.30",86330753,124592,2212989,ever,many


In [44]:
def load_hit_table(adv_set): 
    
    # hit_df = pd.read_pickle(HITS_PATH).filter(regex=r'^[nab].*lower|text|str|head')
    # hit_df = pd.read_pickle(HITS_PATH)
    if HITS_SAMPLE_PATH.is_file(): 
        hit_df = pd.read_pickle(HITS_SAMPLE_PATH)
    else: 
        dfs = [ ]
        for i,p in enumerate(Path('/share/compling/data/sanpi/2_hit_tables/RBXadj/simple').iterdir()):
            if i  > K: 
                break
            _df = pd.read_pickle(p).filter(regex=r'^[nab].*lower|text|str|head')

            _df = _df.loc[_df.adv_form_lower.isin(adv_set), :]
            _df = _df.drop_duplicates()
            word_cols = _df.filter(regex=r'head|lower').columns
            _df[word_cols] = _df[word_cols].astype('category')
            dfs.append(_df)
        hit_df = pd.concat(dfs)
        hit_df.to_pickle(TOP_AM_DIR.joinpath(f'top{K}adv_sample-{K}-hit-tables_{DATE}.pkl.gz'))
    return  hit_df
hits_df = load_hit_table(adv_list)
nb_print_table(hits_df.sort_index(axis=1).sample(13).iloc[:, :4])


|                                            | `adj_form_lower`   | `adv_form_lower`   | `bigrlower`            | `text_window`                                               |
|:-------------------------------------------|:-------------------|:-------------------|:-----------------------|:------------------------------------------------------------|
| **pcc_eng_08_022.2282_x0343802_06:6-7**    | important          | particularly       | particularly_important | on torture is particularly important given President Donald |
| **nyt_eng_19961214_0153_35:31-32**         | important          | particularly       | particularly_important | likely voters was particularly important this year .        |
| **apw_eng_20020713_0384_31:4-5**           | natural            | only               | only_natural           | `` It 's only natural when we are                           |
| **pcc_eng_18_092.4803_x1481380_14:15-16**  | english            | ever               | ever_english           | out t

## Set adverb and collect specific values

In [45]:
def collect_examples(amdf, 
                     hits_df, 
                     adv: str = 'exactly', 
                     n_bigrams: int = BK, 
                     n_examples: int = 50, 
                     metric: str = 'LRC') -> dict:
    df = amdf.copy().filter(like=adv, axis=0).nlargest(n_bigrams, metric)
    examples = {}
    for i, adj in enumerate(df['l2'].unique(), start=1):
        bigram = f'{adv}_{adj}'
        print(f'\n{i}. {bigram}')
        examples[bigram] = sp(
            data=hits_df, print_sample=False, 
            sample_size=n_examples, quiet=True,
            filters=[f'bigram_lower=={bigram}'],
            columns=['bigram_lower', 'text_window', 'token_str'])
        print('   > ', examples[bigram].sample(1).token_str.squeeze())
    return examples

In [46]:
def sample_adv_bigrams(adverb, am_df, hits_df):
    print(f'\n## *{adverb}*\n')
    output_dir = TOP_AM_DIR / 'any_bigram_examples' / adverb
    confirm_dir(output_dir)
    
    this_adv_am = am_df.filter(like=adverb, axis=0).nlargest(BK, 'LRC')
    table_csv_path = output_dir / f'{adverb}_top{BK}-bigrams_AMscores_{timestamp_today()}.csv'
    this_adv_am.to_csv(table_csv_path)
    nb_print_table(this_adv_am, n_dec=2, outpath=table_csv_path.with_suffix('.md'))
    
    examples = collect_examples(this_adv_am, hits_df, adv=adverb, metric='LRC')
    

    print(f'\nSaving Samples in {output_dir}/...')
    paths = []
    for key, df in examples.items(): 
        out_path = output_dir.joinpath(f'{key}_50ex.csv')
        df.to_csv(out_path)
        paths.append(out_path)
    print_iter(paths, header='\nSamples saved as...', bullet='+')
    


In [48]:
adv_am

,key_SET,f_SET,dP1_SET,LRC_SET,G2_SET,N_SET,f1_SET,f2_SET,exp_f_SET,unexp_f_SET,...,mean_G2,mean_N,mean_f1,mean_f2,mean_expF,mean_unexpF,ratio_f_MIR,ratio_N_MIR,ratio_f1_MIR,ratio_f2_MIR
adv,,,,,,,,,,,,,,,,,,,,,
exactly,NEGany~exactly,43635,0.67,5.90,"214,404.20",86330752,3226213,61599,"2,301.98","41,333.02",...,"108,171.83","44,181,417.00","1,760,088.00","31,356.50","1,231.57","20,992.43",0.02,0.02,0.09,0.02
necessarily,NEGany~necessarily,42708,0.72,6.23,"219,003.46",86330752,3226213,56694,"2,118.68","40,589.32",...,"110,346.18","44,181,417.00","1,760,088.00","29,187.50","1,180.93","20,658.57",0.02,0.02,0.09,0.03
before,NEG~before,311,0.38,3.65,"1,062.13",86330752,3226213,748,27.95,283.05,...,"1,071.32","44,181,417.00","1,760,088.00",521.00,35.24,265.26,0.93,0.02,0.09,0.39
that,NEGany~that,165411,0.63,5.62,"781,016.11",86330752,3226213,250392,"9,357.24","156,053.76",...,"394,324.16","44,181,417.00","1,760,088.00","128,932.00","5,219.08","79,655.42",0.03,0.02,0.09,0.03
remotely,NEGany~remotely,5679,0.22,3.03,"13,354.33",86330752,3226213,22194,829.40,"4,849.60",...,"8,682.08","44,181,417.00","1,760,088.00","12,455.50",611.22,"3,151.28",0.33,0.02,0.09,0.12
yet,NEGany~yet,52546,0.48,4.74,"209,055.78",86330752,3226213,101707,"3,800.83","48,745.17",...,"104,649.01","44,181,417.00","1,760,088.00","51,308.00","1,966.16","24,466.84",0.01,0.02,0.09,0.01
ever,NEGany~ever,5967,0.01,0.28,353.58,86330752,3226213,124592,"4,656.05","1,310.95",...,"7,846.96","44,181,417.00","1,760,088.00","64,885.50","2,702.62","2,639.88",0.79,0.02,0.09,0.04
immediately,NEGany~immediately,57319,0.52,4.96,"239,462.58",86330752,3226213,103177,"3,855.76","53,463.24",...,"119,821.89","44,181,417.00","1,760,088.00","52,309.50","2,032.18","26,830.82",0.01,0.02,0.09,0.01
any,NEGany~any,15492,0.13,2.28,"23,683.00",86330752,3226213,94152,"3,518.50","11,973.50",...,"13,097.13","44,181,417.00","1,760,088.00","47,833.00","1,868.76","6,418.24",0.07,0.02,0.09,0.02


In [49]:
adv_adj_am

,f,exp_f,unexp_f,dP1,LRC,G2,N,f1,f2,l1,l2
key,,,,,,,,,,,
yet~unborn,463,0.75,462.25,0.73,10.47,"5,505.48",86330753,101707,635,yet,unborn
only~half-joking,209,1.17,207.83,0.96,9.48,"2,116.01",86330753,464174,217,only,half-joking
inherently~governmental,295,0.60,294.40,0.32,8.96,"3,178.72",86330753,55088,933,inherently,governmental
immediately~accretive,243,0.68,242.32,0.43,8.65,"2,499.70",86330753,103177,565,immediately,accretive
exactly~alike,3040,9.46,"3,030.54",0.23,8.55,"29,939.32",86330753,61599,13261,exactly,alike
...,...,...,...,...,...,...,...,...,...,...,...
only~high,327,"3,177.51","-2,850.51",-0.00,-2.86,"-4,245.38",86330753,464174,590978,only,high
only~sure,339,"4,543.20","-4,204.20",-0.01,-3.34,"-6,708.28",86330753,464174,844981,only,sure
ever~many,206,"3,193.77","-2,987.77",-0.00,-3.42,"-4,923.30",86330753,124592,2212989,ever,many


In [51]:
for adverb in adv_am.index: 
    sample_adv_bigrams(adverb, adv_adj_am, hits_df)


## *exactly*


|                       |   `f` |   `exp_f` |   `unexp_f` |   `dP1` |   `LRC` |      `G2` |        `N` |   `f1` |    `f2` | `l1`    | `l2`      |
|:----------------------|------:|----------:|------------:|--------:|--------:|----------:|-----------:|-------:|--------:|:--------|:----------|
| **exactly~alike**     | 3,040 |      9.46 |    3,030.54 |    0.23 |    8.55 | 29,939.32 | 86,330,753 | 61,599 |  13,261 | exactly | alike     |
| **exactly~opposite**  |   498 |      6.71 |      491.29 |    0.05 |    5.94 |  3,337.27 | 86,330,753 | 61,599 |   9,404 | exactly | opposite  |
| **exactly~right**     | 6,948 |    145.97 |    6,802.03 |    0.03 |    5.53 | 41,085.55 | 86,330,753 | 61,599 | 204,572 | exactly | right     |
| **exactly~zero**      |   344 |      8.19 |      335.81 |    0.03 |    5.02 |  1,912.07 | 86,330,753 | 61,599 |  11,472 | exactly | zero      |
| **exactly~parallel**  |   224 |      5.41 |      218.59 |    0.03 |    4.90 |  1,238.35 | 86,330,753 | 61,